In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.svm import SVR
import plotly.graph_objs as go
import plotly.figure_factory as ff

In [2]:
# Importing dataset and examining it
dataset3 = pd.read_csv("kanpur.csv")

# Data Cleaning and Exploration

In [3]:
pd.set_option('display.max_columns', None) # to make sure you can see all the columns in output window
dataset3.head()

,date_time,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,uvIndex.1,moon_illumination,moonrise,moonset,sunrise,sunset,DewPointC,FeelsLikeC,HeatIndexC,WindChillC,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph
0,2009-01-01 00:00:00,24,10,0.0,8.7,4,1,31,09:56 AM,09:45 PM,06:57 AM,05:28 PM,2,11,12,11,21,17,50,0.0,1015,11,10,320,10
1,2009-01-01 01:00:00,24,10,0.0,8.7,4,1,31,09:56 AM,09:45 PM,06:57 AM,05:28 PM,3,12,13,12,22,11,52,0.0,1015,11,10,315,11
2,2009-01-01 02:00:00,24,10,0.0,8.7,4,1,31,09:56 AM,09:45 PM,06:57 AM,05:28 PM,4,12,13,12,23,6,55,0.0,1015,11,10,310,11
3,2009-01-01 03:00:00,24,10,0.0,8.7,4,1,31,09:56 AM,09:45 PM,06:57 AM,05:28 PM,5,12,13,12,23,0,57,0.0,1015,10,10,304,12
4,2009-01-01 04:00:00,24,10,0.0,8.7,4,1,31,09:56 AM,09:45 PM,06:57 AM,05:28 PM,5,14,14,14,19,0,54,0.0,1016,11,10,302,11


In [4]:
print(dataset3.shape)
print("\n")
print(dataset3.info())
print("\n")
print(dataset3.describe(include = 'all'))

(96432, 25)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96432 entries, 0 to 96431
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date_time          96432 non-null  object 
 1   maxtempC           96432 non-null  int64  
 2   mintempC           96432 non-null  int64  
 3   totalSnow_cm       96432 non-null  float64
 4   sunHour            96432 non-null  float64
 5   uvIndex            96432 non-null  int64  
 6   uvIndex.1          96432 non-null  int64  
 7   moon_illumination  96432 non-null  int64  
 8   moonrise           96432 non-null  object 
 9   moonset            96432 non-null  object 
 10  sunrise            96432 non-null  object 
 11  sunset             96432 non-null  object 
 12  DewPointC          96432 non-null  int64  
 13  FeelsLikeC         96432 non-null  int64  
 14  HeatIndexC         96432 non-null  int64  
 15  WindChillC         96432 non-null  int64  
 16  WindGust

#### Dealing with 'date_time' column

In [5]:
dataset3['date_time'] = pd.to_datetime(dataset3['date_time'])

print(dataset3.info())
print("######################################################")
dataset3['yr'] = dataset3['date_time'].dt.year
dataset3['month'] = dataset3['date_time'].dt.month
dataset3['day'] = dataset3['date_time'].dt.day
dataset3['hour'] = dataset3['date_time'].dt.hour

print(dataset3.info())
pd.set_option('display.max_columns', None) # to make sure you can see all the columns in output window
print(dataset3.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96432 entries, 0 to 96431
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   date_time          96432 non-null  datetime64[ns]
 1   maxtempC           96432 non-null  int64         
 2   mintempC           96432 non-null  int64         
 3   totalSnow_cm       96432 non-null  float64       
 4   sunHour            96432 non-null  float64       
 5   uvIndex            96432 non-null  int64         
 6   uvIndex.1          96432 non-null  int64         
 7   moon_illumination  96432 non-null  int64         
 8   moonrise           96432 non-null  object        
 9   moonset            96432 non-null  object        
 10  sunrise            96432 non-null  object        
 11  sunset             96432 non-null  object        
 12  DewPointC          96432 non-null  int64         
 13  FeelsLikeC         96432 non-null  int64         
 14  HeatIn

In [6]:
# set the "date_time" column as the index
dataset3.set_index('date_time', inplace=True)
print(dataset3.info())
print(dataset3.head())
print("\n")
print(dataset3.shape)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 96432 entries, 2009-01-01 00:00:00 to 2020-01-01 23:00:00
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   maxtempC           96432 non-null  int64  
 1   mintempC           96432 non-null  int64  
 2   totalSnow_cm       96432 non-null  float64
 3   sunHour            96432 non-null  float64
 4   uvIndex            96432 non-null  int64  
 5   uvIndex.1          96432 non-null  int64  
 6   moon_illumination  96432 non-null  int64  
 7   moonrise           96432 non-null  object 
 8   moonset            96432 non-null  object 
 9   sunrise            96432 non-null  object 
 10  sunset             96432 non-null  object 
 11  DewPointC          96432 non-null  int64  
 12  FeelsLikeC         96432 non-null  int64  
 13  HeatIndexC         96432 non-null  int64  
 14  WindChillC         96432 non-null  int64  
 15  WindGustKmph       96432 non-null  

#### Dropping Unnecessary columns

In [7]:
dataset3.columns

Index(['maxtempC', 'mintempC', 'totalSnow_cm', 'sunHour', 'uvIndex',
       'uvIndex.1', 'moon_illumination', 'moonrise', 'moonset', 'sunrise',
       'sunset', 'DewPointC', 'FeelsLikeC', 'HeatIndexC', 'WindChillC',
       'WindGustKmph', 'cloudcover', 'humidity', 'precipMM', 'pressure',
       'tempC', 'visibility', 'winddirDegree', 'windspeedKmph', 'yr', 'month',
       'day', 'hour'],
      dtype='object')

In [8]:
dataset3.drop(['moonrise', 'moonset', 'sunrise', 'sunset', 'moon_illumination',], axis = 1, inplace = True)
dataset3.shape

(96432, 23)

In [9]:
dataset3.columns

Index(['maxtempC', 'mintempC', 'totalSnow_cm', 'sunHour', 'uvIndex',
       'uvIndex.1', 'DewPointC', 'FeelsLikeC', 'HeatIndexC', 'WindChillC',
       'WindGustKmph', 'cloudcover', 'humidity', 'precipMM', 'pressure',
       'tempC', 'visibility', 'winddirDegree', 'windspeedKmph', 'yr', 'month',
       'day', 'hour'],
      dtype='object')

In [10]:
dataset3.isna().sum()

maxtempC         0
mintempC         0
totalSnow_cm     0
sunHour          0
uvIndex          0
uvIndex.1        0
DewPointC        0
FeelsLikeC       0
HeatIndexC       0
WindChillC       0
WindGustKmph     0
cloudcover       0
humidity         0
precipMM         0
pressure         0
tempC            0
visibility       0
winddirDegree    0
windspeedKmph    0
yr               0
month            0
day              0
hour             0
dtype: int64

#### Dividing dataset into label and feature sets and Normalizing numerical features

In [11]:
# Dividing dataset into label and feature sets
X_unscaled = dataset3.drop(['uvIndex', 'yr', 'month', 'day', 'hour', 'humidity'], axis = 1) # Features
Y = dataset3['humidity'] # Labels
print(type(X_unscaled))
print(type(Y))
print(X_unscaled.shape)
print(Y.shape)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
(96432, 17)
(96432,)


In [12]:
# Normalizing numerical features so that each feature has mean 0 and variance 1
feature_scaler = StandardScaler()
X = feature_scaler.fit_transform(X_unscaled)

#### Model Training

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 20)

# Running Machine learning algorithms

## Random Forest

In [14]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# define the rolling window size
window_size = 30

# create the time series split object
tscv = TimeSeriesSplit(n_splits=(X.shape[0]-window_size)//1000, test_size=1000)

# create the model
model = RandomForestRegressor()

# initialize arrays to store the predicted and actual values
Y_pred = np.zeros_like(Y)
Y_test = np.zeros_like(Y)

# iterate over each rolling window
for train_index, test_index in tscv.split(X):
    
    # extract the training and test data for this window
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    
    # fit the model to the training data
    model.fit(X_train, Y_train)
    
    # predict on the test set
    Y_pred[test_index] = model.predict(X_test)
    
# evaluate the performance of the model
mse = mean_squared_error(Y, Y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(Y, Y_pred)
r2 = r2_score(Y, Y_pred)

print('Mean Squared Error:', mse)
print('Root Mean Squared Error:', rmse)
print('Mean Absolute Error:', mae)
print('R-squared:', r2)


Mean Squared Error: 24.32974531275925
Root Mean Squared Error: 4.93251916496624
Mean Absolute Error: 1.6777833084453293
R-squared: 0.9518765625865269


In [15]:
# calculate MAPE
mape = np.mean(np.abs((Y - Y_pred) / Y)) * 100

print('Mean Absolute Percentage Error:', mape)

Mean Absolute Percentage Error: 3.945883618058231


In [17]:
n = X_test.shape[0] # number of samples in test set
p = X_test.shape[1] # number of predictors in the model
r2 = 0.9518765625865269
adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)

print('R-squared:', r2)
print('Adjusted R-squared:', adjusted_r2)

R-squared: 0.9518765625865269
Adjusted R-squared: 0.9510434684561511


## XGBoost Regression

In [18]:
from sklearn.model_selection import TimeSeriesSplit
from xgboost import XGBRegressor

# define the rolling window size
window_size = 30

# create the time series split object
tscv = TimeSeriesSplit(n_splits=(X.shape[0]-window_size)//1000, test_size=1000)

# create the model
model = XGBRegressor()

# initialize arrays to store the predicted and actual values
Y_pred = np.zeros_like(Y)
Y_test = np.zeros_like(Y)

# iterate over each rolling window
for train_index, test_index in tscv.split(X):
    
    # extract the training and test data for this window
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    
    # fit the model to the training data
    model.fit(X_train, Y_train)
    
    # predict on the test set
    Y_pred[test_index] = model.predict(X_test)
    
# evaluate the performance of the model
mse = mean_squared_error(Y, Y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(Y, Y_pred)
r2 = r2_score(Y, Y_pred)

# calculate MAPE
mape = np.mean(np.abs((Y - Y_pred) / Y)) * 100

print('Mean Squared Error:', mse)
print('Root Mean Squared Error:', rmse)
print('Mean Absolute Error:', mae)
print('R-squared:', r2)
print('Mean Absolute Percentage Error:', mape)

Mean Squared Error: 23.199736601957856
Root Mean Squared Error: 4.8166104889183075
Mean Absolute Error: 1.6700369172059069
R-squared: 0.9541116827150723
Mean Absolute Percentage Error: 4.007701316940575


In [21]:
n = X_test.shape[0] # number of samples in test set
p = X_test.shape[1] # number of predictors in the model
r2 = 0.9541116827150723
adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)

print('R-squared:', r2)
print('Adjusted R-squared:', adjusted_r2)

R-squared: 0.9541116827150723
Adjusted R-squared: 0.9533172821103434


## K-Nearest Neighbors regression

In [20]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# define the rolling window size
window_size = 30

# create the time series split object
tscv = TimeSeriesSplit(n_splits=(X.shape[0]-window_size)//1000, test_size=1000)

# create the model
model = KNeighborsRegressor()

# initialize arrays to store the predicted and actual values
Y_pred = np.zeros_like(Y)
Y_test = np.zeros_like(Y)

# iterate over each rolling window
for train_index, test_index in tscv.split(X):
    
    # extract the training and test data for this window
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    
    # fit the model to the training data
    model.fit(X_train, Y_train)
    
    # predict on the test set
    Y_pred[test_index] = model.predict(X_test)
    
# evaluate the performance of the model
mse = mean_squared_error(Y, Y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(Y, Y_pred)
r2 = r2_score(Y, Y_pred)

# calculate MAPE
mape = np.mean(np.abs((Y - Y_pred) / Y)) * 100

print('Mean Squared Error:', mse)
print('Root Mean Squared Error:', rmse)
print('Mean Absolute Error:', mae)
print('R-squared:', r2)
print('Mean Absolute Percentage Error:', mape)

Mean Squared Error: 65.9424050107848
Root Mean Squared Error: 8.120492904423031
Mean Absolute Error: 4.863095238095238
R-squared: 0.869568087966536
Mean Absolute Percentage Error: 11.944470346773297


In [22]:
n = X_test.shape[0] # number of samples in test set
p = X_test.shape[1] # number of predictors in the model
r2 = 0.869568087966536
adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)

print('R-squared:', r2)
print('Adjusted R-squared:', adjusted_r2)

R-squared: 0.869568087966536
Adjusted R-squared: 0.8673101017093374


## Multilayer Perceptron

In [25]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# define the MLPRegressor with a higher maximum number of iterations
model = MLPRegressor(hidden_layer_sizes=(100, 50),
                     activation='relu',
                     solver='adam',
                     max_iter=1000,
                     random_state=42)

# split the data using time series cross-validation
tscv = TimeSeriesSplit(n_splits=5)
# mse_list = []
# rmse_list = []
# mae_list = []
# mape_list = []
# r2_list = []

for train_index, test_index in tscv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]

    # fit the model to the training data
    model.fit(X_train, Y_train)

    # predict on the test set
    Y_pred = model.predict(X_test)

# calculate the evaluation metrics
mse = mean_squared_error(Y_test, Y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(Y_test, Y_pred)
mape = np.mean(np.abs((Y_test - Y_pred) / Y_test)) * 100
r2 = r2_score(Y_test, Y_pred)

# mse_list.append(mse)
# rmse_list.append(rmse)
# mae_list.append(mae)
# mape_list.append(mape)
# r2_list.append(r2)

print('Mean Squared Error:', mse)
print('Root Mean Squared Error:', rmse)
print('Mean Absolute Error:', mae)
print('R-squared:', r2)
print('Mean Absolute Percentage Error:', mape)

Mean Squared Error: 5.9894873428170605
Root Mean Squared Error: 2.4473429148399006
Mean Absolute Error: 1.416552986884023
R-squared: 0.9874084139749414
Mean Absolute Percentage Error: 3.55611842398664


In [26]:
n = X_test.shape[0] # number of samples in test set
p = X_test.shape[1] # number of predictors in the model
r2 = 0.9874084139749414
adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)

print('R-squared:', r2)
print('Adjusted R-squared:', adjusted_r2)

R-squared: 0.9874084139749414
Adjusted R-squared: 0.9873950804155527


## Elastic Net Regression

In [23]:
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import TimeSeriesSplit

# set the train-test split size and random state
test_size = 0.3
random_state = 20

# create a time series split object
tscv = TimeSeriesSplit(n_splits=int((1-test_size)*100))

# specify the Elastic Net hyperparameters
alpha = 0.5
l1_ratio = 0.5

# create the Elastic Net model
model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio)

# iterate through the time series splits
for train_index, test_index in tscv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]

    # fit the model to the training data
    model.fit(X_train, Y_train)

    # predict on the test set
    Y_pred = model.predict(X_test)

# evaluate the performance of the model
mse = mean_squared_error(Y_test, Y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(Y_test, Y_pred)
r2 = r2_score(Y_test, Y_pred)
    
# calculate MAPE
# mape = np.mean(np.abs((Y - Y_pred) / Y)) * 100

print('Mean Squared Error:', mse)
print('Root Mean Squared Error:', rmse)
print('Mean Absolute Error:', mae)
print('R-squared:', r2)
#print('Mean Absolute Percentage Error:', mape)


Mean Squared Error: 43.79513479771057
Root Mean Squared Error: 6.617789268155232
Mean Absolute Error: 5.060605158011421
R-squared: 0.7042806565204585


In [24]:
n = X_test.shape[0] # number of samples in test set
p = X_test.shape[1] # number of predictors in the model
r2 = 0.7042806565204585
adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)

print('R-squared:', r2)
print('Adjusted R-squared:', adjusted_r2)

R-squared: 0.7042806565204585
Adjusted R-squared: 0.7005289932076584
